In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle

In [6]:
selection_strategy = "selected_5"
# read all the folders in the selection strategy
strategy_names = os.listdir(os.path.join("output", selection_strategy))

In [7]:
output_file = os.path.join("output", "selected_5", "BuyAndHoldStrategy", "2004-01-01_2024-01-01.pkl")
with open(output_file, "rb") as f:
    all_results = pickle.load(f)

In [8]:
loop = tqdm(strategy_names)
for strategy_name in loop:  
    loop.set_description(f"Processing {strategy_name}")
    output_file = os.path.join("output", selection_strategy, strategy_name, "2004-01-01_2024-01-01.pkl")
    with open(output_file, "rb") as f:
        all_results = pickle.load(f)
    # calculate the average return
    avg_annual_return = 0
    avg_annual_volatility = 0
    avg_sharpe_ratio = 0
    avg_max_drawdown = 0
    
    # level 0 keys
    rolling_windows = all_results.keys()
    # level 1 keys
    tickers = all_results[list(rolling_windows)[-1]].keys()
    
    results_df_by_tickers = pd.DataFrame(columns=["Period", "ticker", "annual_return (%)", "annual_volatility (%)", "sharpe_ratio", "max_drawdown"])
    
    for ticker in tickers:
        valid_window = 0
        for window in rolling_windows:
            if ticker not in all_results[window]:
                continue
            avg_annual_return += all_results[window][ticker]["annual_return"]
            avg_annual_volatility += all_results[window][ticker]["annual_volatility"]
            avg_sharpe_ratio += all_results[window][ticker]["sharpe_ratio"]
            avg_max_drawdown += all_results[window][ticker]["max_drawdown"]
            valid_window += 1
            results_df_by_tickers = results_df_by_tickers._append(
                {
                    "Period": window, 
                    "ticker": ticker, 
                    "annual_return (%)": round(all_results[window][ticker]["annual_return"] * 100, 4), 
                    "annual_volatility (%)": round(all_results[window][ticker]["annual_volatility"] * 100, 4),
                    "sharpe_ratio": round(all_results[window][ticker]["sharpe_ratio"], 4), 
                    "max_drawdown": round(all_results[window][ticker]["max_drawdown"], 4),
                 }, 
                ignore_index=True)
    
        avg_annual_return /= valid_window
        avg_annual_volatility /= valid_window
        avg_sharpe_ratio /= valid_window
        avg_max_drawdown /= valid_window
        
        results_df_by_tickers = results_df_by_tickers._append(
            {
                "Period": "Average", 
                "ticker": ticker, 
                "annual_return (%)": round(avg_annual_return * 100, 4), 
                "annual_volatility (%)": round(avg_annual_volatility * 100, 4),
                "sharpe_ratio": round(avg_sharpe_ratio, 4), 
                "max_drawdown": round(avg_max_drawdown, 4),
            }, 
            ignore_index=True)
    
    results_df_by_tickers.to_csv(os.path.join("output", selection_strategy, strategy_name, "results.csv"), index=False)

Processing WMAStrategy: 100%|██████████| 5/5 [00:00<00:00, 28.88it/s]           
